In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

In [ ]:
!unrar x /content/gdrive/MyDrive/denemekaps3.rar

In [ ]:
from google.colab import files
files.upload()

In [ ]:

import keras.backend as K
import tensorflow as tf
from keras import initializers, layers
from keras.layers import *


class Length(layers.Layer):
    def call(self, inputs, **kwargs):
        return K.sqrt(K.sum(K.square(inputs), -1))

    def compute_output_shape(self, input_shape):
        return input_shape[:-1]


class Mask(layers.Layer):
    def call(self, inputs, **kwargs):
        if type(inputs) is list:
            assert len(inputs) == 2
            inputs, mask = inputs
            mask = K.expand_dims(mask, -1)
        else:
            x = K.sqrt(K.sum(K.square(inputs), -1, True))
            x = (x - K.max(x, 1, True)) / K.epsilon() + 1
            mask = K.clip(x, 0, 1)

        return K.batch_flatten(inputs * mask)
    def compute_output_shape(self, input_shape):
        if type(input_shape[0]) is tuple:
            return tuple([None, input_shape[0][1] * input_shape[0][2]])
        else:
            return tuple([None, input_shape[1] * input_shape[2]])


def squash(vectors, axis=-1):
    s_squared_norm = K.sum(K.square(vectors), axis, keepdims=True)
    scale = s_squared_norm / (1 + s_squared_norm) / K.sqrt(s_squared_norm + K.epsilon())
    return scale * vectors


class CapsuleLayer(layers.Layer):
    def __init__(self, num_capsule, dim_capsule, num_routing=3,
                 kernel_initializer='glorot_uniform',
                 **kwargs):
        super(CapsuleLayer, self).__init__(**kwargs)
        self.num_capsule = num_capsule
        self.dim_capsule = dim_capsule
        self.num_routing = num_routing
        self.kernel_initializer = initializers.get(kernel_initializer)

    def build(self, input_shape):
        assert len(input_shape) >= 3, "Giriş Tensorunun olması gereken boyutu shape=[None, input_num_capsule, input_dim_capsule]"
        self.input_num_capsule = input_shape[1]
        self.input_dim_capsule = input_shape[2]

        # Matris Dönüştürme
        self.W = self.add_weight(shape=[self.num_capsule, self.input_num_capsule,
                                        self.dim_capsule, self.input_dim_capsule],
                                 initializer=self.kernel_initializer,
                                 name='W')

        self.built = True

    def call(self, inputs, training=None):
      inputs_expand = tf.expand_dims(inputs, 1)
      inputs_tiled  = tf.tile(inputs_expand, [1, self.num_capsule, 1, 1])
      inputs_tiled  = tf.expand_dims(inputs_tiled, 4)

      inputs_hat = tf.map_fn(lambda x: tf.matmul(self.W, x), elems=inputs_tiled)

      b = tf.zeros(shape=[tf.shape(inputs_hat)[0], self.num_capsule,
                      self.input_num_capsule, 1, 1])

      assert self.num_routing > 0, 'The routings should be > 0.'
      for i in range(self.num_routing):

          c = layers.Softmax(axis=1)(b)

          outputs = tf.multiply(c, inputs_hat)
          outputs = tf.reduce_sum(outputs, axis=2, keepdims=True)
          outputs = squash(outputs, axis=-2)

          if i < self.num_routing - 1:
             outputs_tiled = tf.tile(outputs, [1, 1, self.input_num_capsule, 1, 1])
             agreement = tf.matmul(inputs_hat, outputs_tiled, transpose_a=True)
             b = tf.add(b, agreement)

      outputs = tf.squeeze(outputs, [2, 4])
      return outputs

    def compute_output_shape(self, input_shape):
        return tuple([None, self.num_capsule, self.dim_capsule])

def PrimaryCap(inputs, dim_capsule, n_channels, kernel_size, strides, padding):
#    inputTensor=Input(inputs)
    output = layers.Conv2D(filters=dim_capsule*n_channels, kernel_size=kernel_size, strides=strides, padding=padding,
                           name='primarycap_conv2d')(inputs)
    outputs = layers.Reshape(target_shape=[-1, dim_capsule], name='primarycap_reshape')(output)
    return layers.Lambda(squash, name='primarycap_squash')(outputs)




In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import csv
import math
import pandas

def plot_log(filename, show=True):

    data = pandas.read_csv(filename)

    fig = plt.figure(figsize=(4,6))
    fig.subplots_adjust(top=0.95, bottom=0.05, right=0.95)
    fig.add_subplot(211)
    for key in data.keys():
        if key.find('loss') >= 0 and not key.find('val') >= 0:  # training loss
            plt.plot(data['epoch'].values, data[key].values, label=key)
    plt.legend()
    plt.title('Training loss')

    fig.add_subplot(212)
    for key in data.keys():
        if key.find('acc') >= 0:  # acc
            plt.plot(data['epoch'].values, data[key].values, label=key)
    plt.legend()
    plt.title('Training and validation accuracy')

    # fig.savefig('result/log.png')
    if show:
        plt.show()
        plt.savefig('acc_vs_epochs.png')


def combine_images(generated_images, height=None, width=None):
    num = generated_images.shape[0]
    if width is None and height is None:
        width = int(math.sqrt(num))
        height = int(math.ceil(float(num)/width))
    elif width is not None and height is None:  # height not given
        height = int(math.ceil(float(num)/width))
    elif height is not None and width is None:  # width not given
        width = int(math.ceil(float(num)/height))

    shape = generated_images.shape[1:3]
    image = np.zeros((height*shape[0], width*shape[1]),
                     dtype=generated_images.dtype)
    for index, img in enumerate(generated_images):
        i = int(index/width)
        j = index % width
        image[i*shape[0]:(i+1)*shape[0], j*shape[1]:(j+1)*shape[1]] = \
            img[:, :, 0]
    return image

if __name__=="__main__":
    plot_log('log.csv')


In [ ]:
import numpy as np
from keras import layers, models, optimizers
from keras import backend as K
from keras.utils import to_categorical
from keras.layers import *
from keras.utils import plot_model
import matplotlib.pyplot as plt
import cv2
import os
import splitfolders
import pandas as pd
from sklearn import datasets, linear_model
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
from skimage.color import rgb2gray
K.set_image_data_format('channels_last')
from tensorflow.keras.layers import Input, Dense, Lambda
y_pred=[]


def margin_loss(y_true, y_pred):

    L = y_true * K.square(K.maximum(0., 0.9 - y_pred)) + \
        0.5 * (1 - y_true) * K.square(K.maximum(0., y_pred - 0.1))

    return K.mean(K.sum(L, 1))


In [ ]:
import numpy as np
from keras import layers, models, optimizers, callbacks
def train_generator(x, y, batch_size, shift_fraction=0.):
    train_datagen = ImageDataGenerator(
        width_shift_range=shift_fraction,
        height_shift_range=shift_fraction
    )

    generator = train_datagen.flow(x, y, batch_size=batch_size)

    def gen():
        while True:
            x_batch, y_batch = next(generator)
            yield [x_batch, y_batch], [y_batch, x_batch]

    return gen

In [ ]:
def train(model, data, args):
    (x_train, y_train), (x_test, y_test) = data

    # Model ağırlıkları için dosya adı
    weight_path = "{}_capsnet.weights.h5".format('BEST')

    log = callbacks.CSVLogger('log.csv')
    tb = callbacks.TensorBoard(
        log_dir='./tensorboard-logs',
        histogram_freq=args.debug,
        write_graph=True,
        write_images=True
    )
    checkpoint = callbacks.ModelCheckpoint(
        weight_path,
        monitor='capsnet_accuracy',
        save_best_only=True,
        save_weights_only=True,
        verbose=1
    )
    lr_decay = callbacks.LearningRateScheduler(
        schedule=lambda epoch: args.lr * (0.9 ** epoch)
    )
    lr = args.lr

    model.compile(
        optimizer=optimizers.Adam(learning_rate=lr),
        loss=[margin_loss, 'mse'],
        loss_weights=[1., args.lam_recon],
        metrics={'capsnet': 'accuracy'}
    )
    model.fit(
            [x_train, y_train],
            [y_train, x_train],
            batch_size=args.batch_size,
            epochs=args.epochs,
            validation_data=[[x_test, y_test], [y_test, x_test]],
            callbacks=[log, tb, checkpoint]
        )

    # Modeli kaydetme
    model.save_weights('trained_model.weights.h5')
    print("Eğitilmiş model 'trained_model.weights.h5' olarak kaydedildi.")

    return model

In [ ]:
from tensorflow.keras.layers import Lambda
import tensorflow.keras.backend as K

def CapsNet(input_shape, n_class, num_routing):
    x = layers.Input(shape=input_shape)

    conv1 = layers.Conv2D(filters=256, kernel_size=5, strides=1, padding='valid', activation='relu', name='conv1')(x)

    primarycaps = PrimaryCap(conv1, dim_capsule=8, n_channels=32, kernel_size=5, strides=2, padding='valid')

    digitcaps = CapsuleLayer(num_capsule=n_class, dim_capsule=16, num_routing=num_routing, name='digitcaps')(primarycaps)

    # KATMAN 4: Lambda Katmanı ile Normalizasyon
    #normalized_output = Lambda(lambda x: K.l2_normalize(x, axis=-1), output_shape=(n_class, 16), name="normalize_caps")(digitcaps)

    out_caps = Length(name='capsnet')(digitcaps)

    # Kodçözücü Ağ
    y = layers.Input(shape=(n_class,))
    masked_by_y = Mask()([digitcaps, y])
    masked = Mask()(digitcaps)

    decoder = models.Sequential(name='decoder')
    decoder.add(layers.Dense(512, activation='relu', input_dim=16 * n_class))
    decoder.add(layers.Dense(1024, activation='relu'))
    decoder.add(layers.Dense(np.prod(input_shape), activation='sigmoid'))
    decoder.add(layers.Reshape(target_shape=input_shape, name='out_recon'))

    # Eğitim ve Değerlendirme için Modeller
    train_model = models.Model([x, y], [out_caps, decoder(masked_by_y)])
    eval_model = models.Model(x, [out_caps, decoder(masked)])

    return train_model, eval_model


In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

def test(model, data):
    x_test, y_test = data
    y_pred, x_recon = model.predict(x_test, batch_size=2)

    # Test Accuracy
    accuracy = np.sum(np.argmax(y_pred, 1) == np.argmax(y_test, 1)) / y_test.shape[0]
    print('-'*50)
    print(f'Test accuracy: {accuracy}')

    # Precision, Recall, F1 Score
    y_pred_classes = np.argmax(y_pred, axis=1)
    y_true_classes = np.argmax(y_test, axis=1)

    precision = precision_score(y_true_classes, y_pred_classes, average='binary')  # binary classification
    recall = recall_score(y_true_classes, y_pred_classes, average='binary')  # binary classification
    f1 = f1_score(y_true_classes, y_pred_classes, average='binary')  # binary classification

    print(f'Precision: {precision}')
    print(f'Recall: {recall}')
    print(f'F1 Score: {f1}')

    # Save and visualize reconstructed images
    import matplotlib.pyplot as plt
    from PIL import Image

    img = combine_images(np.concatenate([x_test[:50], x_recon[:50]]))
    image = img * 255
    Image.fromarray(image.astype(np.uint8)).save("real_and_recon.png")
    print()
    print('Reconstructed images are saved to ./real_and_recon.png')
    print('-'*50)
    plt.imshow(plt.imread("real_and_recon.png"))
    plt.show()

    return y_pred

In [ ]:
from sklearn.model_selection import KFold
import numpy as np
import cv2
import os
from tensorflow.keras.models import load_model
from keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Hiperparametrelerin Ayarlanması
import argparse
parser = argparse.ArgumentParser()
parser.add_argument('--batch_size', default=4, type=int)
parser.add_argument('--epochs', default=50, type=int)
parser.add_argument('--lam_recon', default=4.92, type=float)
parser.add_argument('--num_routing', default=3, type=int)
parser.add_argument('--shift_fraction', default=0.1, type=float)
parser.add_argument('--debug', default=0, type=int)
parser.add_argument('--save_dir', default='./result')
parser.add_argument('--is_training', default=1, type=int)
parser.add_argument('--weights', default=None)
parser.add_argument('--lr', default=0.0001, type=float)
args, unknown = parser.parse_known_args()

if not os.path.exists(args.save_dir):
    os.makedirs(args.save_dir)

yol = '/content/denemekaps/'
path = yol
data = []
label = []
Files = ['herniyok', 'resim2']
label_val = 0

for files in Files:
    cpath = os.path.join(path, files)
    for img in sorted(os.listdir(cpath)):
        image_array = cv2.imread(os.path.join(cpath, img), cv2.IMREAD_COLOR)
        image_array = cv2.resize(image_array, (320, 320))
        image_array = np.uint8(image_array)
        image_array = cv2.cvtColor(image_array, cv2.COLOR_BGR2GRAY)
        data.append(image_array)
        label.append(label_val)
    label_val = 1

data = np.asarray(data)
label = np.asarray(label, dtype=np.uint8)

kf = KFold(n_splits=5, shuffle=True, random_state=42)

def create_model():
    model, eval_model = CapsNet(input_shape=x_train.shape[1:],
                                n_class=len(np.unique(np.argmax(y_train, 1))),
                                num_routing=args.num_routing)
    return model, eval_model

for fold, (train_index, val_index) in enumerate(kf.split(data)):
    print(f"\nStarting fold {fold + 1}...")

    x_train, x_test = data[train_index], data[val_index]
    y_train, y_test = label[train_index], label[val_index]

    x_train = x_train.reshape(-1, 320, 320, 1).astype('float32') / 255.
    x_test = x_test.reshape(-1, 320, 320, 1).astype('float32') / 255.
    y_train = to_categorical(y_train.astype('float32'))
    y_test = to_categorical(y_test.astype('float32'))

    model, eval_model = create_model()
    model.summary()

    callbacks_list = [
        EarlyStopping(monitor='val_loss', patience=3, verbose=1, restore_best_weights=True),
        ModelCheckpoint(os.path.join(args.save_dir, f"best_model_fold{fold + 1}.keras"),
                        monitor='val_loss', save_best_only=True, verbose=1)
    ]
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=args.lr),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
    x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=42)
    train(model=model, data=((x_train, y_train),(x_val,y_val)), args=args)


    print(f"Testing on fold {fold + 1}...")
    model.load_weights(f"trained_model.weights.h5")
    ypred=test(model=eval_model, data=(x_test, y_test))

